# Create Geohash DB, Search for Geohashes, and Map them 

This Notebook creates a geohash Cloudant Database that groups signal Strenghs together

Import or install libraries uses in notebook

In [1]:
import sys
import time
import pandas as pd

# import Cloudant Library. Install if necessary
try:
    import cloudant
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre cloudant 
    else:
        !pip install --user --pre cloudant
    import cloudant

# import geoJson Library. Install if necessary
try:
    import geojson
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre geojson 
    else:
        !pip install --user --pre geojson
    import geojson

# import geohash Library. Install if necessary
try:
    import geohash2 as Geohash
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre geohash2 
    else:
        !pip install --user --pre geohash2
    import geohash2 as Geohash

# import a second python-geohash Library,. Install if necessary
try:
    Geohash2 = __import__("geohash")
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre python_geohash 
    else:
        !pip install --user --pre python_geohash
    Geohash2 = __import__("geohash")

# import folium mapping Library. Install if necessary
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install --user folium
    import folium

In [2]:
!pip list

branca (0.2.0)
cloudant (2.6.0)
docutils (0.14)
folium (0.5.0)
geohash2 (1.1)
Library (0.0.0)
lucene-querybuilder (0.2)
python-geohash (0.8.5)


Credentials to access the GeoJSON database in Cloudant

In [3]:
# The code was removed by DSX for sharing.

In [4]:
signal_db = 'rf-checker-geojson'
geohash_db="rf-geohash"
rf_credentials = {
    "cloudantNoSQLDB": [
        {
            "credentials": credentials_1,
            "syslog_drain_url": '/dev/null',
            "label": "cloudantNoSQLDB",
            "provider": 'IBM',
            "plan": "Lite",
            "name": "Cloudant NoSQL DB"
        }
    ]
}

Using Curl

In [5]:
import requests

uri= '/rf-checker-geojson/_index'
url= credentials_1['url'] + uri

res = requests.get(url)
res.text

'{"total_rows":1,"indexes":[{"ddoc":null,"name":"_all_docs","type":"special","def":{"fields":[{"_id":"asc"}]}}]}\n'

Some utility functions for later

In [6]:
# Design document used in Cloudant for all indexes
ddoc_name = 'geosignal'

# remove the annoying 'doc' tag from the rows
def removeDocTag(queryResult):
    if len(queryResult['rows']) > 0 and 'doc' in queryResult['rows'][0]:
        queryResult['rows'] = [item['doc'] for item in queryResult['rows']]
    return queryResult
        
# Retrieves documents based on a view and a key
def getViewResults(db, index, key, limit):
    if len(key) > 1:
        queryResult = db.get_view_result(ddoc_name, index, keys=keys, group=True, include_docs=True, limit=limit, raw_result=True)
    else:
        queryResult = db.get_view_result(ddoc_name, index, key=keys[0], include_docs=True, limit=limit, raw_result=True)
    return removeDocTag(queryResult)

# Retrieves documents based on an index and a query
def getSearchResults(db, index, query, bookmark, limit):
    if bookmark == '':
        queryResult = db.get_search_result(ddoc_name, index, query=query, include_docs=True, limit=limit)
    else:
        queryResult = db.get_search_result(ddoc_name, index, query=query, include_docs=True, limit=limit, bookmark=bookmark)
    return removeDocTag(queryResult)

# pauses the thread. This is useful to keep the queroes/second < 5 to Cloudant
# The caller needs to send thew appropriate value of NumCalls
def pauseCalls(numCalls):
    # The free version of cloudant limits the number of call/sec to 5
    numCalls = numCalls + 1
    if numCalls % 4 == 0:
        time.sleep(2)
    return numCalls

if we use a bounding box using 7 digit geohash we get a 153 meter square of data points ==> for my data that is geohash 9xj3gej


Useful GeoJson Utility functions

In [7]:
import statistics as Math

def geoJsonFeature(geojsonStruct, properties, featureId):      
    feature = geojson.Feature(geometry=geojsonStruct, properties=properties)
    feature['id'] = featureId
    return feature

def getProperties(geojsonFeatures, key):
    signalStrengthMean = 0
    strengths = [{'signalStrength': 0,'count': 0, 'dates': []}]
    total = 0
    if len(geojsonFeatures) > 0:
        signalStrengths = [ x['properties']['signalStrength'] for x in geojsonFeatures ]
        signalStrengthMean = Math.mean(signalStrengths)
        total = len(geojsonFeatures)
        # get a list of signal strengths
        strengths = []
        for x in set(signalStrengths):
            dates= [ x['properties']['timestamp'] for x in geojsonFeatures if x['properties']['signalStrength'] == x ]
            strengths.append({ 
                    'signalStrength': x, 
                    'count': signalStrengths.count(x),
                    'dates': dates
            })
    return {
        'signalStrengths': strengths,
        'signalStrengthMean': signalStrengthMean,
        'total': total
    }
           
def convertToGeoJsonMultiPoint(geojsonRows,key):
    return geoJsonFeature(
            geojson.MultiPoint([ item['geometry']['coordinates'] for item in geojsonRows ]),
            getProperties(rows,key),
            "MultiPoint")
    
def convertToGeoJsonPolygon(geojsonRows, key):
    feature = geoJsonFeature(
            geojson.Polygon([[ item['geometry']['coordinates'] for item in geojsonRows ]]), 
            getProperties(geojsonRows,key),
            "Polygon")

def bboxAsLatLong(bbox):
    rectangle = []
    for x in [['w', 'n'], ['e', 'n'], ['e', 's'], ['w', 's'], ['w', 'n']]:
        rectangle.append([bbox[x[0]], bbox[x[1]]])
    return rectangle 

def convertToGeohashRectangle(geojsonRows, key):
    geohashbox = Geohash2.bbox(key)
    return geoJsonFeature(
            geojson.Polygon([bboxAsLatLong(geohashbox)]),
            getProperties(geojsonRows,key),
            key)

def _neighbors(nearestNeighbors):
    allNeighbors = set()
    for neighbor in nearestNeighbors:
        nearestNeighbors2 = set(Geohash2.neighbors(neighbor))
        nearestNeighbors2 = nearestNeighbors2 - allNeighbors
        allNeighbors = allNeighbors | nearestNeighbors2
    return allNeighbors

def getGeohashNeighbors(geohashToSearchFor,numBoxes):
    # uses sets to manage duplicate geohashes
    allNeighbors = set([geohashToSearchFor])   
    nearestNeighbors = allNeighbors
    count = 0
    for x in range(0,numBoxes):
        nearestNeighbors = _neighbors(nearestNeighbors)
        nearestNeighbors = nearestNeighbors - allNeighbors
        allNeighbors = allNeighbors | nearestNeighbors
    return list(allNeighbors)

# this routine will find the the middle of the geohash points lat and long
def findFeaturesCenter(features):
    # Routine find the average lat and long in the features
    pointsList= []
    for feature in features:
        points = feature['geometry']['coordinates'][0][0:4]
        pointsList.extend(points)
    lat = Math.mean([x[1] for x in pointsList])
    long = Math.mean([x[0] for x in pointsList])
    return [lat, long]
            
def findGeohashCenter(geohash):
    geohashCenter = Geohash.decode_exactly(geohash)
    # decode_exactly gives us 4 values. We need the first 2
    centerOfMap = [ x for x in geohashCenter[:2] ] 
    return centerOfMap

# Removes a signal Strength from the list of features and returns the resulting list
def removeSignalStrength(features, dbm):
    return [item for item in features if item['properties']['signalStrengthMean'] != dbm]

Example of how to find all the Feature Points for a geohash list

if we find a geohash feature for geohash code then continue
else search for all feature points for the geohash code, create a geohash Feature for the pointd and store it in DB for later use

In this way the first time the geohash is called it takes a short time to "create" the geohash feature, but after trhat it is really quick

In [8]:
# Used to keep the number of calls to Cloudant in the range of 5/sec

# This will search for geohash features in either signal or geohash DB
def _searchForGeohashFeatures(db, selector, fields, sort):
    numCalls = 1
    rows = []
    page_size=200
    query = cloudant.query.Query(db,selector=selector, fields=fields)
    with query.custom_result(page_size=page_size) as result:
        count = 0
        page=[]
        while True:
            page = [item for item in result[count :count + page_size]]
            count = count + page_size
            rows.extend(page)
            if len(page) < page_size:
                break;
            numCalls= pauseCalls(numCalls)
    return rows

def _searchGeohashDB(db, selector):
    fields = ["geometry", "properties", "id", "type"]
    sort= [{"id": "asc"}]
    features = _searchForGeohashFeatures(db, selector, fields, sort)
    return features

# finds the list of points in the signals DB for the geohashes and inserts them into the geohash DB
def _populateMissingGeohashFeatures(db,dbf,missingGeohashes):
    missingGeohashFeatures = []
    if len(missingGeohashes) > 0:
        # find all the points in the signal DB
        selector = {
            "geohash": {"$in": missingGeohashes},
            'id': {"$eq": 'signal-strength'}
        }
        fields= ["geometry", "properties", "geohash", "id", "type"]
        sort= [{"_id": "asc"}]
        pointFeatures = _searchForGeohashFeatures(db, selector, fields, sort)       
        # group them into geohashes
        dict = {}
        for group in missingGeohashes:
            points = [item for item in pointFeatures if group in item['geohash']]
            dict[group] = points
        # now create the geohash Feature and store it in database
        for key in missingGeohashes:
            points = dict[key]
            missingGeohashFeatures.append(convertToGeohashRectangle(points, key))
        # now store them    
        dbf.bulk_docs(missingGeohashFeatures)
        print("Created " + str(len(missingGeohashFeatures)) + ' missing Geohashes in Geohash DB')
    return missingGeohashFeatures

# find all the geohash features for the list of geohashes
# This will force the geohash DB to be created.
# Note it does not "update" the geohash DB
def populateGeohashDB(geohashes):
    features = []
    with cloudant.cloudant_bluemix(rf_credentials,'Cloudant NoSQL DB') as client:
        # First find if there is a geohash Feature for the geohashes
        # For those that do not exist search for the points that are contained in the missing geohashes and craete a geohash object into DB
        # eventually will need to change this when new Points are added to the points database
        db = client[signal_db]
        dbf = client[geohash_db]
        selector = {
            "id": {"$in": geohashes}
        }
        features = _searchGeohashDB(dbf, selector)
        print ('Found ' + str(len(features)) + ' Geohashes in the Geohash DB')
        # determine which geohashes in list are not in the geohash DB
        missingGeohashes = set(geohashes) - set([item['id'] for item in features])
        newFeatures = _populateMissingGeohashFeatures(db,dbf,list(missingGeohashes))
        features.extend(newFeatures)
    return features

# searches the geohash DB for a feature matching the selector criteria
def searchGeohashDB(selector):
    features = []
    with cloudant.cloudant_bluemix(rf_credentials,'Cloudant NoSQL DB') as client:
        db = client[geohash_db]
        features = _searchGeohashDB(db, selector)
        print ('Found ' + str(len(features)) + ' Geohashes in the Geohash DB')
    return features

In [9]:
import branca.colormap as cm

# finds the lowest and highest signal Strength range in the geoJson features
# N.B. if this routine finds a Zero signalStrength it assumes you want to include that in the range
def findMinMaxStrength(features):
    signalStrengthList = [ item['properties']['signalStrengthMean'] for item in features] if len(features) > 0 else [0]
    # find lowest signal Strength
    low = min(signalStrengthList)
    # find highest signal Strength
    high = max(signalStrengthList)
    return [low, high]

#returned the color palltette tpo be used for the features (it igniores zero signal Strengths)
def getColorScale(features):
    strengthRange = findStrengthRange(features)
    colorScale = cm.linear.PuOr.scale(strengthRange[0], strengthRange[1])
    colorScale.caption='Signal Strength (dbm)'
    return colorScale

Set up Pandas DataFrame for color for geohashes

In [10]:
mygeohash= '9xj3gdy'
mygeohash= '9xj3gdyy'
#mygeohash= '9xj3gejwz'
#mygeohash = '9xj3geq34' # tower hashcode
mygeohash = Geohash.encode(39.699343, -104.955471, precision=8) # rough center of my points

Query to find all signal strengths in a 1km radius from a tower

In [11]:
# finds a list of geohashes around a central geohash to a certain depth
# This will force pre-population of the Geohash database for further queries.
geohashes = getGeohashNeighbors(mygeohash,18)
features_dict = {}
features_dict['All Geohashes'] = populateGeohashDB(geohashes)

Found 0 Geohashes in the Geohash DB
Created 1369 missing Geohashes in Geohash DB


Query to find all "high" signal Strengths

In [12]:
selector = {
    "_id": {"$gt": 0} ,
    "$and": [
        { "properties.signalStrengthMean": { "$gt": -85 } },
        { "properties.signalStrengthMean": { "$lt": -50 } }
    ]
}   
features_dict['High Signal Strength'] = searchGeohashDB(selector)

Found 22 Geohashes in the Geohash DB


Query to find all Low Signal Strengths

In [13]:
selector = {
    "_id": {"$gt": 0} ,
    "properties.signalStrengthMean": { "$lt": -95 }
}   
features_dict['Low Signal Strength'] = searchGeohashDB(selector)

Found 155 Geohashes in the Geohash DB


For all the features create a map with a layer for each feature

Display each feature group in a layer on the map

In [14]:
# remove all zero signal Strengths from features_dict 

# collect all the necessary ranges for displaying on the map
centerOfMapList=[]
colorScaleList = []
for key, features in features_dict.items():
    # removes signal Strength == 0 from feature dictionary
    features_dict[key] = removeSignalStrength(features, 0)
    centerOfMapList.append(findFeaturesCenter(features))
    colorScaleList.append(findMinMaxStrength(features))

# determine center of map for ALL features being displayed
lat = Math.mean([coord[0] for coord in centerOfMapList])
long = Math.mean([coord[1] for coord in centerOfMapList])
centerOfMap = [ lat, long ]

# determine color range for all Signal Strengths being displayed
low = min([strength[0] for strength in colorScaleList if strength[0] < 0])
high= max([strength[1] for strength in colorScaleList if strength[1] < 0])
colorScale = cm.linear.PuOr.scale(low, high)                      
colorScale.caption='Signal Strength (dbm)'
print(centerOfMap)
colorScale

[39.70001210635009, -104.95613060109427]


In [15]:
def style_function(feature):
    opacity=0.7
    color= 'grey'
    geohash= feature['id']
    ss = feature['properties']['signalStrengthMean']
    if ss != 0:
        color = colorScale(ss)
    else:
        opacity=0.05
    return{ 
        'color': 'grey', 'weight':1, 'opacity': opacity,
        'fillColor': color, 'fillOpacity':opacity
    }

def highlight_function(feature):
    return {
        'weight': 5,
        'color': 'black',
        'dashArray': [],
        'fillOpacity': 0.7
    }

map = folium.Map(location=centerOfMap, zoom_start=17, control_scale=True)
colorScale.add_to(map)

for key, features in features_dict.items():
    shape = geojson.FeatureCollection(features)
    g = folium.GeoJson(
        shape,
        name=key,
        control = True,
        overlay=True,  
        style_function=style_function, 
        highlight_function=highlight_function).add_to(map)
    # map.add_child(addHoverover(g))
folium.LayerControl().add_to(map)
map    